### __1. Import and data pre-processing:__

In [ ]:
# import counts dataframe:

In [ ]:
# import metadata:


In [ ]:
# select (x, y) data:
# x -> counts table
# y -> (sample, phenotype, denger)


In [ ]:
# normalization:


In [ ]:
# meta -> (subject_id, sample, phenotype, gender)


In [ ]:
# one-hot encoding:


In [ ]:
# shuffling:


In [ ]:
# splitting (x, y) into train, validation, and test:


In [ ]:
# deleting unnecessary arrays:


### __2. Neural network modeling:__

### __3. Model compilation:__

### __4. Train and validation__   

### __5. Final training__    

### __6. Test evaluation__:

#### __6.1 Anomaly detection:__

### __7. Saving the model__:
<font size=3>
    
For model __loading__, see [2.2-notebook](2.2-notebook.ipynb).